<a href="https://colab.research.google.com/github/A-Peoples/BCS-Recreated/blob/main/BCS_Recreated_1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# installing sportsdataverse
!pip install sportsdataverse
!pip install xgboost==1.6.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 85.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.2/223.2 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.3/418.3 kB 23.6 MB/s eta 0:00:00
  Attempting uninstall: polars
    Found existing installation: polars 1.31.0
    Uninstalling polars-1.31.0:
      Successfully uninstalled polars-1.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-polars-cu12 25.10.0 requires polars<1.33,>=1.28, but you have polars 0.18.15 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.9/255.9 MB 6.3 MB/s eta 0:00:00
  Attempting uninstall: xgboost
    Found existing installation: xgboost 3.1.1
    Uninstalling xgboost-3.1.1:
      Successfully uninstalled xgboost-3.1.1


In [2]:
import sportsdataverse as sdv
import pandas as pd
import numpy as np

In [3]:
data = sdv.cfb.load_cfb_schedule([2025], return_as_pandas=True)
data["margin"] = data["home_points"] - data["away_points"]
team_names = data['home_team'].unique()

data = data.dropna(subset=['home_points', 'away_points'])

data.shape

100%|██████████| 1/1 [00:00<00:00,  2.48it/s]


(3467, 32)

In [4]:
data["margin"] = data["home_points"] - data["away_points"]

final_team = pd.DataFrame()
wins = 0
losses = 0
c_wins = 0
c_losses = 0
margin = 0
total_games = 0
conference = ''
global team_opp_eilo
for team_name in team_names:
  team_opp_eilo = []
  points_scored = 0
  points_against = 0
  eilo = 0
  data["margin"] = data["home_points"] - data["away_points"]
  data
  team = data.loc[(data['home_team'] == team_name) | (data['away_team'] == team_name)]
  team
  main_team = pd.DataFrame()
  for index, row in team.iterrows():
    if row["home_team"] == team_name:
      points_scored = points_scored + row['home_points']
      points_against = points_against + row['away_points']
      total_games +=1
      eilo = eilo + row["home_pregame_elo"]
      team_opp_eilo.append(row["away_pregame_elo"])
      if row["home_points"] >  row["away_points"]:
        if row["conference_game"] == True:
          c_wins +=1
        wins +=1
      elif row["home_points"] <  row["away_points"]:
        if row["conference_game"] == True:
          c_losses +=1
        losses +=1
      margin += row["margin"]
      conference = row['home_conference']
    elif row["away_team"] == team_name:
      points_against = points_against + row['home_points']
      points_scored = points_scored + row['away_points']
      eilo = eilo + row["away_pregame_elo"]
      team_opp_eilo.append(row["home_pregame_elo"])
      total_games +=1
      if row["home_points"] <  row["away_points"]:
        if row["conference_game"] == True:
          c_wins +=1
        wins +=1
      elif row["home_points"] >  row["away_points"]:
        if row["conference_game"] == True:
          c_losses +=1
        losses +=1
      margin += row["margin"]
      conference = row['away_conference']
      points_scored = points_scored / total_games
      points_against = points_against / total_games
  main_team = pd.DataFrame({
      "team": [team_name],
      "wins": [wins],
      "losses": [losses],
      "c_wins": [c_wins * 0.5],
      "c_losses": [c_losses * 1.5],
      'conference': [conference],
      'points_scored': [points_scored],
      'points_against': [points_against],
      "margin": [margin],
      'eilo': [eilo],
      'games_played': [total_games]
      })
  final_team = pd.concat([final_team, main_team], ignore_index=True)
  wins = 0
  losses = 0
  c_wins = 0
  c_losses = 0
  margin = 0
  total_games = 0

In [5]:
FBS = ('SEC', 'Sun Belt', 'Big Ten', 'Pac-12', 'Conference USA', 'FBS Independents', 'ACC', 'Mountain West', 'Big 12', "Mid American")

In [41]:
FBS = final_team.loc[final_team['conference'].isin(['SEC', 'Sun Belt', 'Big Ten', 'Pac-12', 'Conference USA', 'FBS Independents', 'ACC', 'Mountain West', 'Big 12', "Mid American"])]

FBS.eilo = FBS['eilo'] / FBS['games_played']

FBS["points"] = ((FBS['wins'] * 15) - (FBS['losses']) * 15) + ((FBS['c_wins'] * 15) - (FBS['c_losses']) * 15) + (FBS['margin'] / 10) + (FBS['eilo'] / 10)

FBS = FBS.sort_values(by=["points"], ascending=False)
FBS["rank"] = range(1, len(FBS) + 1)
FBS = FBS.head(25)
FBS = FBS[["rank","team","conference", "wins", "losses", "points", 'margin','eilo']]


/tmp/ipython-input-2292179495.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FBS.eilo = FBS['eilo'] / FBS['games_played']
/tmp/ipython-input-2292179495.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FBS["points"] = ((FBS['wins'] * 15) - (FBS['losses']) * 15) + ((FBS['c_wins'] * 15) - (FBS['c_losses']) * 15) + (FBS['margin'] / 10) + (FBS['eilo'] / 10)


**Run This**

In [42]:
FBS["points"]

FBS

,rank,team,conference,wins,losses,points,margin,eilo
105,1,Indiana,Big Ten,11,0,445.572727,222,1983.727273
80,2,Ohio State,Big Ten,10,0,418.51,112,2048.1
157,3,Texas A&M,SEC,10,0,383.74,37,1775.4
166,4,Ole Miss,SEC,10,1,369.527273,173,1947.272727
163,5,Texas Tech,Big 12,10,1,361.027273,200,1760.272727
112,6,Georgia,SEC,9,1,360.66,74,2032.6
119,7,Oregon,Big Ten,9,1,351.91,105,1989.1
131,8,James Madison,Sun Belt,9,1,350.29,99,1678.9
167,9,BYU,Big 12,9,1,323.02,119,1686.2
405,10,Alabama,SEC,8,2,319.42,140,1929.2
